In [32]:
import chess
import numpy

squares_index = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d


In [75]:
import numpy
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	# Download dataset.npz from the link in Readme.md
	container = numpy.load('dataset.npz')
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5,
	                  dtype=numpy.float32)  # normalization (0 - 1)
	return b, v


x_train, y_train = get_dataset()

In [76]:
# Split dataset for two clients
# pip install -U scikit-learn
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_train, y_train, test_size=0.1, random_state=100)

# Initial Model Dataset
x_train, x_train1, y_train, y_train2 = train_test_split(
    x_test, y_test, test_size=0.3, random_state=100)

# Client Dataset
fed_x_train, x_test, fed_y_train, y_test = train_test_split(
    x_train, y_train, test_size=0.1, random_state=100)

fed_x_train, x_test, fed_y_train, y_test = train_test_split(
    x_test, y_test, test_size=0.3, random_state=100
)


In [77]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def build_model(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = board3d
  for _ in range(conv_depth):
    x = layers.Conv2D(filters=conv_size, kernel_size=3,
                      padding='same', activation='relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, 'relu')(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

In [78]:
model = build_model(32, 4)

In [79]:
from tensorflow.keras.callbacks import ModelCheckpoint
model.compile(optimizer=optimizers.Adam(5e-4), loss='mean_squared_error')
# model.summary()

model.fit(x_train1, y_train2,
          batch_size=2048,
          epochs=10,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])

model.save_weights("model_weight/new_cen_weights", save_format="h5")

Epoch 1/10
20/20 [==============================] - 12s 548ms/step - loss: 0.0028 - val_loss: 0.0020
Epoch 2/10
20/20 [==============================] - 12s 613ms/step - loss: 0.0017 - val_loss: 0.0016
Epoch 3/10
20/20 [==============================] - 12s 582ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 4/10
20/20 [==============================] - 11s 574ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 5/10
20/20 [==============================] - 12s 575ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 6/10
20/20 [==============================] - 11s 561ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 7/10
20/20 [==============================] - 11s 567ms/step - loss: 9.8327e-04 - val_loss: 9.8444e-04
Epoch 8/10
20/20 [==============================] - 12s 592ms/step - loss: 8.4499e-04 - val_loss: 9.9065e-04
Epoch 9/10
20/20 [==============================] - 12s 608ms/step - loss: 7.8376e-04 - val_loss: 8.4580e-04
Epoch 10/10
20/20 [==============================] - 12s 591ms/step

In [96]:
fed1 = model
fed2 = model
fed3 = model
fed4 = model


In [119]:
fed1.load_weights("model_weight/new_cen_weights")
fed1.fit(fed_x_train, fed_y_train,
          batch_size=2048,
          epochs=10,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])

fed1_weights = fed1.get_weights()


Epoch 1/10
4/4 [==============================] - 2s 610ms/step - loss: 8.1111e-04 - val_loss: 8.1854e-04
Epoch 2/10
4/4 [==============================] - 2s 619ms/step - loss: 7.9241e-04 - val_loss: 8.0635e-04
Epoch 3/10
4/4 [==============================] - 2s 608ms/step - loss: 7.7386e-04 - val_loss: 8.0086e-04
Epoch 4/10
4/4 [==============================] - 3s 636ms/step - loss: 7.6296e-04 - val_loss: 7.9226e-04
Epoch 5/10
4/4 [==============================] - 3s 641ms/step - loss: 7.5847e-04 - val_loss: 7.9331e-04
Epoch 6/10
4/4 [==============================] - 3s 653ms/step - loss: 7.4657e-04 - val_loss: 7.8254e-04
Epoch 7/10
4/4 [==============================] - 3s 640ms/step - loss: 7.2781e-04 - val_loss: 7.9965e-04
Epoch 8/10
4/4 [==============================] - 3s 627ms/step - loss: 7.2737e-04 - val_loss: 7.8613e-04
Epoch 9/10
4/4 [==============================] - 2s 591ms/step - loss: 7.1956e-04 - val_loss: 7.9686e-04
Epoch 10/10
4/4 [=============================

In [120]:
fed2.load_weights("model_weight/new_cen_weights")
fed2.fit(fed_x_train, fed_y_train,
          batch_size=2048,
          epochs=10,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])
fed2_weights = fed2.get_weights()


Epoch 1/10
4/4 [==============================] - 2s 510ms/step - loss: 8.1053e-04 - val_loss: 6.8932e-04
Epoch 2/10
4/4 [==============================] - 2s 498ms/step - loss: 7.8033e-04 - val_loss: 6.9031e-04
Epoch 3/10
4/4 [==============================] - 2s 494ms/step - loss: 7.7288e-04 - val_loss: 6.7804e-04
Epoch 4/10
4/4 [==============================] - 2s 501ms/step - loss: 7.5032e-04 - val_loss: 6.7530e-04
Epoch 5/10
4/4 [==============================] - 2s 559ms/step - loss: 7.4752e-04 - val_loss: 6.6582e-04
Epoch 6/10
4/4 [==============================] - 2s 534ms/step - loss: 7.3102e-04 - val_loss: 6.5670e-04
Epoch 7/10
4/4 [==============================] - 3s 621ms/step - loss: 7.1383e-04 - val_loss: 6.5210e-04
Epoch 8/10
4/4 [==============================] - 2s 555ms/step - loss: 7.0273e-04 - val_loss: 6.5218e-04
Epoch 9/10
4/4 [==============================] - 2s 565ms/step - loss: 6.9372e-04 - val_loss: 6.4856e-04
Epoch 10/10
4/4 [=============================

In [121]:
client1_weights = numpy.array(fed1_weights, dtype=object)
client2_weights = numpy.array(fed2_weights, dtype=object)
client1_weights = 1/2 * client1_weights
client2_weights = 1/2 * client2_weights
aggregated_weights = numpy.add(client1_weights,client2_weights)

fed1.set_weights(aggregated_weights)
fed1.save_weights("model_weight/new_fed_weights", save_format="h5")

In [122]:
fed3.load_weights("model_weight/new_cen_weights")
fed3.fit(x_train1, y_train2,
         batch_size=2048,
         epochs=10,
         verbose=1,
         validation_split=0.1,
         callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                    callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])
fed3_weights = fed3.get_weights()


Epoch 1/10
20/20 [==============================] - 14s 688ms/step - loss: 7.1649e-04 - val_loss: 8.0116e-04
Epoch 2/10
20/20 [==============================] - 13s 653ms/step - loss: 6.9262e-04 - val_loss: 7.9037e-04
Epoch 3/10
20/20 [==============================] - 13s 661ms/step - loss: 6.7489e-04 - val_loss: 7.5290e-04
Epoch 4/10
20/20 [==============================] - 13s 635ms/step - loss: 6.5606e-04 - val_loss: 7.4143e-04
Epoch 5/10
20/20 [==============================] - 13s 639ms/step - loss: 6.4952e-04 - val_loss: 7.2545e-04
Epoch 6/10
20/20 [==============================] - 12s 623ms/step - loss: 6.4547e-04 - val_loss: 7.1558e-04
Epoch 7/10
20/20 [==============================] - 13s 629ms/step - loss: 6.2870e-04 - val_loss: 7.4997e-04
Epoch 8/10
20/20 [==============================] - 12s 624ms/step - loss: 6.3702e-04 - val_loss: 7.6081e-04
Epoch 9/10
20/20 [==============================] - 13s 643ms/step - loss: 6.2092e-04 - val_loss: 6.8659e-04
Epoch 10/10
20/20 [

In [123]:
fed4.load_weights("model_weight/new_cen_weights")
fed4.fit(x_train1, y_train2,
         batch_size=2048,
         epochs=10,
         verbose=1,
         validation_split=0.1,
         callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                    callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])
fed4_weights = fed4.get_weights()

Epoch 1/10
20/20 [==============================] - 13s 678ms/step - loss: 7.2012e-04 - val_loss: 7.8901e-04
Epoch 2/10
20/20 [==============================] - 14s 685ms/step - loss: 6.8037e-04 - val_loss: 7.7008e-04
Epoch 3/10
20/20 [==============================] - 14s 683ms/step - loss: 6.6813e-04 - val_loss: 7.7182e-04
Epoch 4/10
20/20 [==============================] - 13s 643ms/step - loss: 6.6281e-04 - val_loss: 7.6087e-04
Epoch 5/10
20/20 [==============================] - 13s 670ms/step - loss: 6.6817e-04 - val_loss: 7.2299e-04
Epoch 6/10
20/20 [==============================] - 13s 648ms/step - loss: 6.3937e-04 - val_loss: 7.8590e-04
Epoch 7/10
20/20 [==============================] - 13s 651ms/step - loss: 6.3142e-04 - val_loss: 7.1532e-04
Epoch 8/10
20/20 [==============================] - 13s 643ms/step - loss: 6.3915e-04 - val_loss: 7.0222e-04
Epoch 9/10
20/20 [==============================] - 13s 648ms/step - loss: 6.1232e-04 - val_loss: 6.8025e-04
Epoch 10/10
20/20 [

In [124]:
fed3_weights = fed3.get_weights()
fed4_weights = fed4.get_weights()
client1_weights = numpy.array(fed3_weights, dtype=object)
client2_weights = numpy.array(fed4_weights, dtype=object)
client1_weights = 1/2 * client1_weights
client2_weights = 1/2 * client2_weights
aggregated_weights = numpy.add(client1_weights, client2_weights)
fed3.set_weights(aggregated_weights)
fed3.save_weights("model_weight/new_fed1_weights", save_format="h5")

In [128]:
model.load_weights("model_weight/new_cen_weights")
result0 = model.evaluate(x_test, y_test, batch_size = 32)
print("test loss, test acc:", result0)

99/99 [==============================] - 0s 4ms/step - loss: 8.1669e-04
test loss, test acc: 0.000816692307125777


In [129]:
fed1.load_weights("model_weight/new_fed_weights")
result1 = fed1.evaluate(x_test, y_test, batch_size = 32)
print("test loss, test acc:", result1)

99/99 [==============================] - 0s 4ms/step - loss: 7.6132e-04
test loss, test acc: 0.0007613188354298472


In [130]:
fed3.load_weights("model_weight/new_fed1_weights")
result1 = fed3.evaluate(x_test, y_test, batch_size=32)
print("test loss, test acc:", result1)


99/99 [==============================] - 0s 4ms/step - loss: 7.1827e-04
test loss, test acc: 0.0007182715344242752
